# 🧭 Sistema de Router Chains - Guia de Viagem

**Objetivo**: Implementar um sistema inteligente que classifica consultas turísticas e direciona para chains especializadas.

## 🎯 Tipos de Consulta Suportadas:
1. **`roteiro-viagem`**: Planejamento de roteiros e itinerários
2. **`logistica-transporte`**: Informações sobre transporte e logística
3. **`info-local`**: Dados sobre pontos turísticos, restaurantes, cultura
4. **`traducao-idiomas`**: Tradução e comunicação em outros idiomas

## 📋 Arquitetura:
- **Router Chain**: Classifica a intenção da consulta
- **Chains Especializadas**: Processam cada tipo de consulta
- **RAG Integration**: Usa dados do Pinecone para respostas contextualizadas

In [ ]:
# Imports e configuração inicial
import os
from dotenv import load_dotenv
from pinecone import Pinecone
from langchain_huggingface import HuggingFaceEmbeddings
from langchain_groq import ChatGroq
from langchain.chains import LLMChain
from langchain.chains.router import MultiPromptChain
from langchain.chains.router.llm_router import LLMRouterChain, RouterOutputParser
from langchain.prompts import PromptTemplate
from langchain_core.output_parsers import StrOutputParser

# Carregar variáveis de ambiente
load_dotenv()

print("✅ Imports carregados com sucesso!")

In [ ]:
# Inicializar componentes principais
print("🔄 Inicializando componentes...")

# LLM Groq
llm = ChatGroq(
    temperature=0.1,
    model="llama-3.1-70b-versatile",
    groq_api_key=os.getenv('GROQ_API_KEY')
)

# Pinecone e embeddings (do notebook anterior)
pinecone_client = Pinecone(api_key=os.getenv('PINECONE_API_KEY'))
indice = pinecone_client.Index('guia-viagem')
embeddings_model = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")

print("✅ Componentes inicializados:")
print(f"🤖 LLM: {llm.model_name}")
print(f"🗂️ Pinecone: {indice.describe_index_stats()['total_vector_count']} vetores")
print(f"🎯 Embeddings: {embeddings_model.model_name}")

In [ ]:
# Definir prompts especializados para cada tipo de consulta

# 1. Template para Roteiro de Viagem
roteiro_template = """
Você é um especialista em planejamento de roteiros turísticos. Use o contexto fornecido para criar roteiros detalhados.

CONTEXTO RELEVANTE:
{context}

CONSULTA DO USUÁRIO:
{input}

INSTRUÇÕES:
- Crie um roteiro estruturado e detalhado
- Inclua horários sugeridos e duração das atividades
- Considere proximidade geográfica dos pontos
- Adicione dicas práticas e recomendações

RESPOSTA:
"""

# 2. Template para Logística e Transporte
logistica_template = """
Você é um especialista em logística de viagem e sistemas de transporte urbano. Use o contexto para orientar sobre mobilidade.

CONTEXTO RELEVANTE:
{context}

CONSULTA DO USUÁRIO:
{input}

INSTRUÇÕES:
- Forneça informações detalhadas sobre transporte
- Inclua custos, horários e rotas quando possível
- Sugira alternativas de transporte
- Dê dicas de segurança e eficiência

RESPOSTA:
"""

# 3. Template para Informações Locais
info_local_template = """
Você é um guia turístico local expert. Use o contexto fornecido para dar informações detalhadas sobre pontos turísticos, cultura e recomendações.

CONTEXTO RELEVANTE:
{context}

CONSULTA DO USUÁRIO:
{input}

INSTRUÇÕES:
- Forneça informações detalhadas e precisas
- Inclua dados históricos, culturais ou curiosidades
- Adicione recomendações práticas (horários, preços, dicas)
- Mantenha um tom acolhedor e informativo

RESPOSTA:
"""

# 4. Template para Tradução e Idiomas
traducao_template = """
Você é um assistente de tradução e comunicação intercultural. Ajude com traduções, frases úteis e orientações culturais.

CONTEXTO RELEVANTE:
{context}

CONSULTA DO USUÁRIO:
{input}

INSTRUÇÕES:
- Forneça traduções precisas e contextualizadas
- Inclua pronúncia quando relevante
- Adicione informações sobre etiqueta cultural
- Sugira frases alternativas para diferentes situações

RESPOSTA:
"""

print("✅ Templates de prompt definidos para 4 categorias!")

In [ ]:
# Função para buscar contexto relevante no Pinecone
def buscar_contexto_relevante(consulta, top_k=3):
    """
    Busca informações relevantes no Pinecone baseado na consulta do usuário.
    """
    try:
        # Gerar embedding da consulta
        query_embedding = embeddings_model.embed_documents([consulta])[0]
        
        # Buscar no Pinecone
        resultado = indice.query(
            vector=query_embedding, 
            top_k=top_k, 
            include_metadata=True
        )
        
        # Extrair textos relevantes
        contextos = []
        for match in resultado['matches']:
            if match['score'] > 0.3:  # Filtro de relevância
                contextos.append(match['metadata']['text'])
        
        return "\n\n".join(contextos) if contextos else "Nenhum contexto específico encontrado na base de conhecimento."
    
    except Exception as e:
        print(f"Erro na busca: {e}")
        return "Erro ao buscar contexto na base de conhecimento."

# Testar a função
teste_contexto = buscar_contexto_relevante("pontos turísticos do Rio de Janeiro")
print("🔍 Teste da busca de contexto:")
print(f"Primeiros 200 caracteres: {teste_contexto[:200]}...")

In [ ]:
# Criar chains especializadas com RAG
def criar_chain_com_rag(template, nome):
    """
    Cria uma chain que integra RAG (busca no Pinecone) com o template específico.
    """
    prompt = PromptTemplate(
        template=template,
        input_variables=["input", "context"]
    )
    
    def chain_com_contexto(consulta):
        # Buscar contexto relevante
        contexto = buscar_contexto_relevante(consulta)
        
        # Executar chain com contexto
        chain = LLMChain(llm=llm, prompt=prompt)
        return chain.run(input=consulta, context=contexto)
    
    return chain_com_contexto

# Criar as 4 chains especializadas
chains = {
    "roteiro-viagem": criar_chain_com_rag(roteiro_template, "Roteiro de Viagem"),
    "logistica-transporte": criar_chain_com_rag(logistica_template, "Logística e Transporte"),
    "info-local": criar_chain_com_rag(info_local_template, "Informações Locais"),
    "traducao-idiomas": criar_chain_com_rag(traducao_template, "Tradução e Idiomas")
}

print("✅ Chains especializadas criadas:")
for nome in chains.keys():
    print(f"📋 {nome}")

In [ ]:
# Criar o Router Chain para classificar consultas
router_template = """
Você é um classificador de consultas turísticas. Analise a consulta do usuário e determine qual categoria ela pertence.

CATEGORIAS DISPONÍVEIS:
- roteiro-viagem: Planejamento de roteiros, itinerários, programação de atividades
- logistica-transporte: Transporte, como chegar, mobilidade urbana, horários
- info-local: Informações sobre pontos turísticos, restaurantes, cultura, história
- traducao-idiomas: Tradução, frases úteis, comunicação, idiomas

CONSULTA: {input}

INSTRUÇÕES:
- Responda APENAS com o nome da categoria
- Use exatamente os nomes: roteiro-viagem, logistica-transporte, info-local, ou traducao-idiomas
- Se não tiver certeza, use 'info-local' como padrão

CATEGORIA:
"""

router_prompt = PromptTemplate(
    template=router_template,
    input_variables=["input"]
)

router_chain = LLMChain(llm=llm, prompt=router_prompt)

print("✅ Router Chain criado para classificação de consultas!")

In [ ]:
# Sistema principal - Guia de Viagem Inteligente
def guia_viagem_inteligente(consulta_usuario):
    """
    Sistema principal que processa consultas turísticas:
    1. Classifica a consulta
    2. Direciona para a chain apropriada
    3. Retorna resposta contextualizada com RAG
    """
    print(f"🔍 Processando consulta: '{consulta_usuario}'")
    
    try:
        # Passo 1: Classificar a consulta
        categoria = router_chain.run(consulta_usuario).strip().lower()
        print(f"📂 Categoria identificada: {categoria}")
        
        # Passo 2: Verificar se a categoria é válida
        if categoria not in chains:
            print(f"⚠️ Categoria '{categoria}' não reconhecida. Usando 'info-local' como padrão.")
            categoria = "info-local"
        
        # Passo 3: Executar a chain especializada
        print(f"🤖 Executando chain especializada: {categoria}")
        resposta = chains[categoria](consulta_usuario)
        
        # Passo 4: Retornar resultado estruturado
        resultado = {
            "consulta": consulta_usuario,
            "categoria": categoria,
            "resposta": resposta
        }
        
        return resultado
        
    except Exception as e:
        print(f"❌ Erro no processamento: {e}")
        return {
            "consulta": consulta_usuario,
            "categoria": "erro",
            "resposta": f"Desculpe, ocorreu um erro ao processar sua consulta: {e}"
        }

print("✅ Sistema Guia de Viagem Inteligente configurado!")
print("🎯 Pronto para processar consultas turísticas!")

In [ ]:
# Testes do sistema com diferentes tipos de consulta
consultas_teste = [
    "Quais são os principais pontos turísticos do Rio de Janeiro?",
    "Como posso criar um roteiro de 2 dias em Paris?",
    "Como chegar do aeroporto ao centro do Rio?",
    "Como dizer 'onde fica o banheiro?' em francês?"
]

print("🧪 Iniciando testes do sistema...\n")

for i, consulta in enumerate(consultas_teste, 1):
    print(f"\n{'='*60}")
    print(f"TESTE {i}:")
    print(f"{'='*60}")
    
    resultado = guia_viagem_inteligente(consulta)
    
    print(f"\n📝 RESULTADO:")
    print(f"📂 Categoria: {resultado['categoria']}")
    print(f"💬 Resposta: {resultado['resposta'][:300]}...")
    print(f"\n{'='*60}")